# GRU

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchmetrics

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\connor\AppData\Local\Programs\Python\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
DATA_PATH = r'C:\Users\connor\PycharmProjects\trading\data\analytics\analytics_voo.csv'
df = pd.read_csv(DATA_PATH, low_memory=False)

In [ ]:
df.columns

In [ ]:
non_target_columns = ['open', 'high', 'low', 'close', 'volume', 'CPALTT01USM657N', 'DFF', 'EXPINF10YR', 'GDPC1', 'RSXFS', 'T10YFF', 'UNRATE', 'macd', 'macd_signal', 'macd_hist', 'daily_obv', 'seven_day_ema', 'close', 'target']

df_cols = df[non_target_columns]

# version with just close price and date
df_target_train = df[['date', 'target']].where(df.date <= '2020-01-01')
df_target_train.dropna(inplace=True)
df_target_test = df[['date', 'target']].where(df.date > '2020-01-01')
df_target_test.dropna(inplace=True)

# version with all columns
df_target_train_v2 = df[non_target_columns].where(df.date <= '2020-01-01')
df_target_train_v2.dropna(inplace=True)
df_target_test_v2 = df[non_target_columns].where(df.date > '2020-01-01')
df_target_test_v2.dropna(inplace=True)

In [ ]:
print(df_target_train.columns)
print()
print(df_target_train_v2.columns)

In [ ]:
def create_sequences(df, seq_length, num_rows=len(df), include_all_features=False):
    """
    Input:
    df: pandas dataframe with at least 2 columns, a date column and a target column
    seq_length: How long the sequence should be. 
    num_rows: How many rows to use. Will pull all rows unless num_rows is provided, in which case the top num_rows will be pulled.
    include_all_features: Whether to use just the target column for the sequence, or to include all features.
        
    Output:
    Generates a sequence of seq_length length. It's shape is [num_rows, seq_length] if include_all_features is False, else [num_rows, seq_length * len(df.columns)]. 
    A sequence refers to how many should be appended as columns. 
    For example, if df = [['2020-01-01',100], ['2020-01-02', 101], ['2020-01-03', 102]] and seq_length = 2, the generated sequence will be [[101,102], [102,103]]. 
    """
    
    
    xs, ys = [], []
    
    if include_all_features == False:
        for itr in range(num_rows - seq_length):
            x = df.iloc[itr:(itr + seq_length), -1]
            y = df.iloc[itr + seq_length, -1]
            xs.append(x)
            ys.append(y)
    elif include_all_features == True:
        for itr in range(num_rows - seq_length):
            x = df.iloc[itr:(itr + seq_length), :]
            y = df.iloc[itr + seq_length, -1]
            xs.append(x)
            ys.append(y)

    else:
        print(f"error: include_all_features accepts True or False, got {include_all_features} instead.")
    return np.array(xs), np.array(ys)

In [ ]:
# create training set
sequence_len = 10
num_rows = 100000
X_train, y_train = create_sequences(df_target_train, sequence_len, num_rows)
X_test, y_test = create_sequences(df_target_test, sequence_len, num_rows)

X_train_v2, y_train_v2 = create_sequences(df_target_train_v2, sequence_len, num_rows, include_all_features=True)
X_test_v2, y_test_v2 = create_sequences(df_target_test_v2, sequence_len, num_rows, include_all_features=True)

In [ ]:
df[non_target_columns].head(n=100)

In [ ]:
X_train[0]

In [ ]:
y_train[0]

In [ ]:
X_train_v2[0]

In [ ]:
y_train_v2[0]

In [ ]:
print(len(X_train), len(X_train_v2))
print(len(y_train), len(y_train_v2))

# using all fields available

In [ ]:
# convert df into objects Torch can read
torch_X_train  = torch.from_numpy(X_train_v2).float()
torch_y_train = torch.from_numpy(y_train_v2).float()
torch_X_test = torch.from_numpy(X_test_v2).float()
torch_y_test = torch.from_numpy(y_test_v2).float()

# normalize data
torch_X_train = torch.nn.functional.normalize(torch_X_train)
torch_X_test = torch.nn.functional.normalize(torch_X_test)

print(torch_X_train.shape, torch_y_train.shape)
# create test and train sets
train_data_set = TensorDataset(torch_X_train, torch_y_train)
test_data_set = TensorDataset(torch_X_test, torch_y_test)

# confirm it works
sample = train_data_set[0]
input_sample, label_sample = sample
print('input sample:', input_sample)
print('label sample:', label_sample)

# Using just the close price

In [ ]:
# # convert df into objects Torch can read
# torch_X_train  = torch.from_numpy(X_train).float()
# torch_y_train = torch.from_numpy(y_train).float()
# torch_X_test = torch.from_numpy(X_test).float()
# torch_y_test = torch.from_numpy(y_test).float()
# 
# # create test and train sets
# train_data_set = TensorDataset(torch_X_train, torch_y_train)
# test_data_set = TensorDataset(torch_X_test, torch_y_test)
# 
# # confirm it works
# sample = train_data_set[0]
# input_sample, label_sample = sample
# print('input sample:', input_sample)
# print('label sample:', label_sample)

In [ ]:
print(input_sample.shape)

In [ ]:
num_features = len(df_target_train_v2.columns)
batch_size = 1000
shuffle = False
hidden_size = round(num_features / 2)
num_layers = 2
dropout = .5

learning_rate = 0.0001
num_epochs = 50

# create dataloader
train_dataloader = DataLoader(train_data_set, batch_size=batch_size, shuffle=shuffle, drop_last=True)
test_dataloader = DataLoader(test_data_set, batch_size=batch_size, shuffle=shuffle, drop_last=True)
# test loader
# x, y = next(iter(train_dataloader))
# 
# print('x', x, 'y', y)

In [ ]:
# GRU
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__() #super makes all the methods available in nn.Module available for the new class Net
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(in_features=hidden_size, out_features=1)
        self.sigmoid = nn.Sigmoid() # we want a binary output, not %
        
    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

In [ ]:
net = Net(input_size=num_features)
criterion = nn.BCELoss(reduction='sum') # for binary prediction. Using the 'target' column
#criterion = nn.MSELoss() # for regression, predicting the 'close' column
optimizer = optim.Adam(
    net.parameters(), lr=learning_rate
)
acc = torchmetrics.Accuracy(task="binary")

for epoch in range(num_epochs):
    for seqs, labels in train_dataloader:
        seqs = seqs.view(batch_size, sequence_len, num_features)
        outputs = net(seqs).squeeze()
        loss = criterion(outputs, labels)
        acc(torch.round(outputs), labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}, Accuracy: {acc.compute()}")
    # early break if loss isn't changing beyond the learning rate
    if loss.item() < .1:
        break

In [ ]:
# # Define MSE metric
# mse = torchmetrics.regression.MeanSquaredError()
# 
# net.eval()
# with torch.no_grad():
#     for seqs, labels in test_dataloader:
#         seqs = seqs.view(batch_size, torch_X_test.shape[1], num_features)
#         # Pass seqs to net and squeeze the result
#         outputs = net(seqs).squeeze()
#         mse(outputs, labels)
# 
# # Compute final metric value
# test_mse = mse.compute()
# print(f"Test MSE: {test_mse}")
# print(f"Test RMSE: {test_mse**.5}")

In [ ]:
# Test set
net.eval()
with torch.no_grad():
    for seqs, labels in test_dataloader:
        seqs = seqs.view(batch_size, sequence_len, num_features)
        outputs = net(seqs).squeeze()
        acc(torch.round(outputs), labels)

print(f"Test accuracy score: {acc.compute()}")



# Results

Baseline guessing 

In [ ]:
random_guess_likelihood = df.target.mean()
# mean = percent the candle was higher than the previous candle

print('Random guess likelihood', random_guess_likelihood)

# Writeup
At the minute level, 46.875% of the time the next close price is higher than the previous close price for VOO.
The model is able to correctly predict a candle 56.201% of the time making it better than a random (weighted) coinflip. The thing holding the model back is that it doesn't take into account expected payoff. The average positive candel is higher than the average negative candle in absolute terms. The model is doing almost exactly as well on the train and test set so there is not any overfitting going on. The accuracy is a bit low though so even though it's a nn it has high bias and underfitting overall. It is just a hair less accurate than the LSTM.


I used the following hyperparameters:
sequence_len = 10
* Sequence of length 10 seemed to give sufficient information without overwhelming my computer. Each input becomes sequence_len * feature so the compute cost is quite high increasing this value
num_rows = 100000
* This is to limit how much data I'm using. Adding more data doesn't seem to improve accuracy but it does crash performance.
batch_size = 1000
* Mostly arbitarily chosen. 1000 makes the model run faster and doesn't seem to sacrifice accuracy either.
shuffle = False
* don't want to shuffle sequence data since that defeats the purpose
hidden_size = round(num_features / 2)
* Heuristic that the best hidden_size is based on the average between the input and output sizes. Here my output is just 1 so it's just the number of features divided by 2
num_layers = 2
* Again heuristic, 2 hidden layers should be able to solve anything
dropout = .5
* Another mostly random chosen value. Playing around with this doesn't impact the model too much.

learning_rate = 0.0001
* small enough to change directions but not too small to where it doesn't update
num_epochs = 50
* Probably don't need this many. It stops really changing much after like 3-4 epochs


I chose not to do dimensionality reduction for the GRU model. I'm allowing the model to figure out which features and relationships are important or not. A big part of the value of deep learning is that it doesn't require me doing any sort of feature engineering or reduction.